# Tick data

For optimum results this notebook should be run during the Forex trading session.

In [ ]:
from ib_async import *

util.startLoop()

ib = IB()
ib.connect("127.0.0.1", 4001, clientId=15)

<IB connected to 127.0.0.1:4001 clientId=15>

Error 317, reqId 6: Market depth data has been RESET. Please empty deep book contents before applying any new entries., contract: Contract(secType='FUT', conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ')
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 317, reqId 6: Market depth data has been RESET. Please empty deep book contents before applying any new entries., contract: Contract(secType='FUT', conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ')
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. The following farms are connected: usfuture; usfarm; secdefil. The following farms are not connected: ushmds.
Error 317, reqId 6: Market depth data has been RESET. Please empty deep bo

In [2]:
contract = Contract(conId=620730920)
ib.qualifyContracts(contract)
ticker = ib.reqMktDepth(contract)

In [3]:
from IPython.display import display, clear_output
import pandas as pd

In [42]:
positions[0].contract.localSymbol

'NQM4'

In [6]:
positions = ib.positions(account="U10394496")

columns = ["Account", "LocalSymbol", "Position", "AvgCost"]
df = pd.DataFrame(columns=columns)
rows = []

for pos in positions:
    new_row = {
        "Account": pos.account,
        "LocalSymbol": pos.contract.localSymbol,
        "Position": pos.position,
        "AvgCost": pos.avgCost,
    }
    rows.append(new_row)

df = pd.DataFrame(rows, columns=columns)
df.set_index("LocalSymbol", inplace=True)

display(df)

,Account,Position,AvgCost
LocalSymbol,,,
HYG 240621P00073000,U10394496,50.0,12.446850
AAPL 241018P00150000,U10394496,2.0,244.696850
NVDA 240719P00555000,U10394496,1.0,493.049400
LLY 240816P00620000,U10394496,1.0,481.046900
GOOGL 240920P00120000,U10394496,10.0,51.446850
NQM4,U10394496,-2.0,349960.446576
QQQ 240607P00400000,U10394496,2.0,207.696850


In [7]:
# Filter orders with filledQuantity equal to 0.0
unfilled_orders = [
    order for order in ib.orders()  # if order.filledQuantity != order.totalQuantity
]

# order by permId
unfilled_orders.sort(key=lambda x: x.permId)

# Iterate over the filtered list and print details
print(f"id\t\taction\tlmtPrice\tfillQty\ttotalQty")
for order in unfilled_orders:

    print(
        f"{order.permId}\t{order.action}\t{order.lmtPrice}\t{order.filledQuantity}\t{order.totalQuantity}\t{order.st}"
    )
    # print(order)

id		action	lmtPrice	fillQty	totalQty
1939819052	BUY	17525.0	0.0	1.0
1939819053	BUY	17524.25	0.0	1.0
1939819054	BUY	17524.0	0.0	1.0
1939819055	BUY	17522.75	0.0	1.0
1939819056	BUY	17522.5	0.0	1.0
1939819057	BUY	17524.5	0.0	1.0
1939819058	BUY	17523.25	0.0	1.0
1939819059	BUY	17524.75	0.0	1.0
1939819060	BUY	17524.0	0.0	1.0
1939819061	BUY	17522.0	0.0	1.0
1939819062	BUY	17521.75	0.0	1.0
1939819063	BUY	17521.25	0.0	1.0
1939819064	BUY	17521.75	0.0	1.0
1939819065	BUY	17522.0	0.0	1.0
1939819066	BUY	17522.0	0.0	1.0
1939819067	BUY	17521.0	1.0	0.0
1939819068	SELL	17523.5	1.0	0.0
1939819069	BUY	17521.0	0.0	1.0
1939819070	BUY	17521.0	0.0	1.0
1939819071	BUY	17519.75	0.0	1.0
1939819072	BUY	17520.75	0.0	1.0
1939819073	BUY	17519.0	0.0	1.0
1939819074	BUY	17517.0	1.0	0.0
1939819076	SELL	17481.5	0.0	1.0
1939819077	SELL	17479.75	1.0	0.0
1939819078	BUY	17477.25	1.0	0.0
1939819079	SELL	17480.0	0.0	1.0
1939819080	SELL	17478.5	1.0	0.0


In [11]:
ib.reqCompletedOrders()

TypeError: IB.reqCompletedOrders() missing 1 required positional argument: 'apiOnly'

In [64]:
orders = ib.orders()
orders[0]

Order(permId=827631043, action='BUY', totalQuantity=1.0, orderType='LMT', lmtPrice=17248.0, auxPrice=0.0, tif='GTC', ocaType=3, orderRef='cLnpF0mGeHCl', displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', cashQty=0.0, dontUseAutoPriceForHedge=True, autoCancelDate='20240930 17:00:00 Central Standard Time', filledQuantity=0.0, refFuturesConId=2147483647, shareholder='Not an insider or substantial shareholder')

In [53]:
limitOrder = LimitOrder("BUY", 20000, 0.05)
limitTrade = ib.placeOrder(contract, limitOrder)

limitTrade

AttributeError: 'list' object has no attribute 'updateEvent'

In [14]:
# long scalp strategy
placed = False
target_pos = 5
for pos in positions:
    if pos.contract.conId == contract.conId:
        print(f"position: {pos.position}, target: {target_pos}")
        if placed == False and pos.position < target_pos:
            ib.placeOrder(contract, Order(action="BUY", totalQuantity=1, orderType="MKT"))

current: 4.0, target: 5


### Streaming tick data

Create some Forex contracts:

In [3]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

Wait a few seconds for the tickers to get filled.

In [4]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2019, 12, 31, 17, 5, 2, 127038, tzinfo=datetime.timezone.utc), bid=1.12245, bidSize=10700000, ask=1.1225, askSize=2000000, high=1.1239, low=1.11985, close=1.12, halted=0.0)

The price of Forex ticks is always nan. To get a midpoint price use ``midpoint()`` or ``marketPrice()``.

The tickers are kept live updated, try this a few times to see if the price changes:

In [5]:
ticker.marketPrice()

1.1224750000000001

The following cell will start a 30 second loop that prints a live updated ticker table.
It is updated on every ticker change.

In [6]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(
    index=[c.pair() for c in contracts],
    columns=['bidSize', 'bid', 'ask', 'askSize', 'high', 'low', 'close'])

def onPendingTickers(tickers):
    for t in tickers:
        df.loc[t.contract.pair()] = (
            t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)        

ib.pendingTickersEvent += onPendingTickers
ib.sleep(30)
ib.pendingTickersEvent -= onPendingTickers

,bidSize,bid,ask,askSize,high,low,close
EURUSD,11500000,1.1225,1.12255,1000000,1.1239,1.11985,1.12
USDJPY,8000000,108.665,108.675,16000000,108.885,108.475,108.87
GBPUSD,6000000,1.327,1.3271,500000,1.32845,1.3106,1.3111
USDCHF,10000000,0.9677,0.9678,7000000,0.9698,0.96465,0.96935
USDCAD,1000000,1.2966,1.29665,2000000,1.30695,1.29515,1.3068
AUDUSD,9500000,0.70305,0.7031,1000000,0.70325,0.69945,0.6995


New tick data is available in the 'ticks' attribute of the pending tickers.
The tick data will be cleared before the next update.

To stop the live tick subscriptions:

In [7]:
for contract in contracts:
    ib.cancelMktData(contract)

### Tick by Tick data ###

The ticks in the previous section are time-sampled by IB in order to cut on bandwidth. So with ``reqMktdData`` not every tick from the exchanges is sent. The promise of ``reqTickByTickData`` is to send every tick, just how it appears in the TWS Time & Sales window. This functionality is severly nerfed by a total of just three simultaneous subscriptions, where bid-ask ticks and sale ticks also use up a subscription each.

The tick-by-tick updates are available from ``ticker.tickByTicks`` and are signalled by ``ib.pendingTickersEvent`` or ``ticker.updateEvent``.

In [8]:
ticker = ib.reqTickByTickData(eurusd, 'BidAsk')
ib.sleep(2)
print(ticker)

ib.cancelTickByTickData(ticker.contract, 'BidAsk')

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2019, 12, 31, 17, 5, 35, 432737, tzinfo=datetime.timezone.utc), bid=1.1225, bidSize=11000000, ask=1.1226, askSize=11500000, prevBid=1.12255, prevBidSize=11500000, prevAsk=1.12255, prevAskSize=1000000, high=1.1239, low=1.11985, close=1.12, halted=0.0, tickByTicks=[TickByTickBidAsk(time=datetime.datetime(2019, 12, 31, 17, 5, 35, 432737, tzinfo=datetime.timezone.utc), bidPrice=1.1225, askPrice=1.12255, bidSize=11500000, askSize=1000000, tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False)), TickByTickBidAsk(time=datetime.datetime(2019, 12, 31, 17, 5, 35, 432737, tzinfo=datetime.timezone.utc), bidPrice=1.1225, askPrice=1.12255, bidSize=11000000, askSize=1000000, tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False)), TickByTickBidAsk(time=datetime.datetime(2019, 12, 31, 17, 5, 35, 432737, tzinfo=datetime.timezone.utc)

### Historical tick data

Historical tick data can be fetched with a maximum of 1000 ticks at a time. Either the start time or the end time must be given, and one of them must remain empty:

In [9]:
import datetime

start = ''
end = datetime.datetime.now()
ticks = ib.reqHistoricalTicks(eurusd, start, end, 1000, 'BID_ASK', useRth=False)

ticks[-1]

HistoricalTickBidAsk(time=datetime.datetime(2019, 12, 31, 17, 5, 34, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False), priceBid=1.1225, priceAsk=1.1226, sizeBid=11000000, sizeAsk=11500000)

In [10]:
ib.disconnect()